## <font color='darkblue'>Preface</font>
([article source](https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/)) <font size='3ptx'><b>Hyperparameter optimization is a big part of deep learning.</b> The reason is that neural networks are notoriously difficult to configure and there are a lot of parameters that need to be set. On top of that, individual models can be very slow to train.</font>

In this post you will discover how you can use the grid search capability from the scikit-learn python machine learning library to tune the hyperparameters of Keras deep learning models. After reading this post you will know:
* How to wrap Keras models for use in scikit-learn and how to use grid search.
* How to grid search common neural network parameters such as learning rate, dropout rate, epochs and number of neurons.
* How to define your own hyperparameter tuning experiments on your own projects.

<a id='sect0'></a>
### <font color='darkgreen'>Overview</font>
In this post, I want to show you both how you can use the scikit-learn grid search capability and give you a suite of examples that you can copy-and-paste into your own project as a starting point.

Below is a list of the topics we are going to cover:
1. <font size='3ptx'><b><a href='#sect1'>How to use Keras models in scikit-learn.</a></b></font>
2. <font size='3ptx'><b><a href='#sect2'>How to use grid search in scikit-learn.</a></b></font>
3. <font size='3ptx'><b><a href='#sect3'>How to tune batch size and training epochs.</a></b></font>
4. How to tune optimization algorithms.
5. How to tune learning rate and momentum.
6. How to tune network weight initialization.
7. How to tune activation functions.
8. How to tune dropout regularization.
9. How to tune the number of neurons in the hidden layer.

<a id='sect1'></a>
## <font color='darkblue'>How to Use Keras Models in scikit-learn</font>
<font size='3ptx'><b>Keras models can be used in scikit-learn by wrapping them with the <font color='blue'>KerasClassifier</font> or <font color='blue'>KerasRegressor</font> class from the module [SciKeras](https://pypi.org/project/scikeras/)</b>. You may need to run the command <font color='blue'>pip install scikeras</font> first to install the module.</font>

In [3]:
#!pip install scikeras

To use these wrappers you must define a function that creates and returns your Keras sequential model, then pass this function to the model argument when constructing the <b><font color='blue'>KerasClassifier</font></b> class. For example:
```python
def create_model():
	...
	return model

model = KerasClassifier(model=create_model)
```
<br/>

The constructor for the <b><font color='blue'>KerasClassifier</font></b> class can also take new arguments that can be passed to your custom <font color='blue'>create_model()</font> function. These new arguments must also be defined in the signature of your <font color='blue'>create_model()</font> function with default parameters:
```python
def create_model(dropout_rate=0.0):
	...
	return model

model = KerasClassifier(model=create_model, dropout_rate=0.2)
```
<br/>

You can learn more about these from the [SciKeras documentation](https://www.adriangb.com/scikeras/stable/index.html).

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier

print(tf.__version__)

2.9.1


<a id='sect2'></a>
## <font color='darkblue'>How to Use Grid Search in scikit-learn</font>
<font size='3ptx'><b>Grid search is a model hyperparameter optimization technique.</b> In scikit-learn this technique is provided in the [**GridSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) class.</font>

When constructing this class you must provide a dictionary of hyperparameters to evaluate in the <font color='violet'>param_grid</font> argument. This is a map of the model parameter name and an array of values to try.

By default, accuracy is the score that is optimized, but other scores can be specified in the <font color='violet'>score</font> argument of the [**GridSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) constructor.

<b>By default, the grid search will only use one thread. By setting the <font color='violet'>n_jobs</font> argument in the [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) constructor to -1, the process will use all cores on your machine</b>. However, sometimes this may interfere with the main neural network training process.

The [**GridSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) process will then construct and evaluate one model for each combination of parameters. <b>Cross validation is used to evaluate each individual model and the default of 3-fold cross validation is used</b>, although this can be overridden by specifying the <font color='violet'>c</font>v argument to the [**GridSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) constructor.

Below is an example of defining a simple grid search:
```python
param_grid = dict(epochs=[10,20,30])
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
```
<br/>

Once completed, you can access the outcome of the grid search in the result object returned from <font color='blue'>grid.fit()</font>. The `best_score_` member provides access to the best score observed during the optimization procedure and the `best_params_` describes the combination of parameters that achieved the best results.

You can learn more about the [GridSearchCV class in the scikit-learn API documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

## <font color='darkblue'>Problem Description</font> ([back](#sect0))
<font size='3ptx'><b>Now that we know how to use Keras models with scikit-learn and how to use grid search in scikit-learn, let’s look at a bunch of examples.</b> All examples will be demonstrated on a small standard machine learning dataset called the [Pima Indians onset of diabetes classification dataset](https://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes). This is a small dataset with all numerical attributes that is easy to work with.</font> 

[Download the dataset](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv) and place it in path <font color='olive'>datas/kaggle_pima-indians-diabetes-database/diabetes.csv</font>.

<b>As we proceed through the examples in this post, we will aggregate the best parameters</b>. This is not the best way to grid search because parameters can interact, but it is good for demonstration purposes.

In [3]:
# load dataset
dataset = pd.read_csv("../../datas/kaggle_pima-indians-diabetes-database/diabetes.csv")

<a id='sect3'></a>
## <font color='darkblue'>How to Tune Batch Size and Number of Epochs</font> ([back](#sect0))
<font size='3ptx'><b>In this first simple example, we look at tuning the batch size and number of epochs used when fitting the network.</b></font>

The **batch size** in [iterative gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Iterative_method) is the number of patterns shown to the network before the weights are updated. It is also an optimization in the training of the network, defining how many patterns to read at a time and keep in memory.

The **number of epochs** is the number of times that the entire training dataset is shown to the network during training. Some networks are sensitive to the batch size, such as LSTM recurrent neural networks and Convolutional Neural Networks.

Here we will evaluate a suite of different mini batch sizes from 10 to 100 in steps of 20.

The full code listing is provided below.

In [5]:
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_shape=(8,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model